## DATA READING

In [0]:
df = spark.read.format("parquet")\
               .option("inferSchema", "true")\
               .load("abfss://bronze@adlscardatalake.dfs.core.windows.net/raw_data/")

In [0]:
display(df)

## DATA TRANSFORMATION

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import*

In [0]:
df = df.withColumn('Model_Category',split(col('Model_ID'), '-')[0])
display(df)

In [0]:
df = df.withColumn("RevenuePerUnit", col('Revenue')/col('Units_Sold'))
df.display()

In [0]:
display(df.groupBy('Year','BranchName').agg(sum('Units_Sold').alias('Total_Sales')).sort('Year','Total_Sales',ascending=[1,0]))

Databricks visualization. Run in Databricks to view.

## DATA WRITING

In [0]:
df.write.format('parquet').mode('overwrite').option("path","abfss://silver@adlscardatalake.dfs.core.windows.net/carsales/").save()

## QUERYING SILVER DATA

In [0]:
%sql
SELECT * FROM parquet.`abfss://silver@adlscardatalake.dfs.core.windows.net/carsales/`